<a href="https://colab.research.google.com/github/rvraghvender/DeepLearningProjects/blob/main/NaturalLanguageProcessing/WordVector/Operation_on_word_vectors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Operatiosn on Word Vectors

**In this project, we will**
- Explain how word embeddings capture relationships between words
- Measure similarity between word vectors using cosine similarity
- Use word embeddings to solve word analogy problems such as Man is to Woman, as King is to ____.
- Finally, we will modify words embeddings to reduce their gender bias.


In [1]:
import urllib.request
import collections
import os
import zipfile
import numpy as np

import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Reshape
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import skipgrams
from tensorflow.keras.preprocessing import sequence


print("Version: ", tf.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

Version:  2.13.0
GPU is NOT AVAILABLE


In [2]:
window_size = 3
vector_dim = 300
epochs = 1000

valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)

In [3]:
def maybe_download(filename, url, expected_bytes):
    """Download a file if not present, and make sure it's the right size."""
    if not os.path.exists(filename):
        filename, _ = urllib.request.urlretrieve(url + filename, filename)
    statinfo = os.stat(filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified', filename)
    else:
        print(statinfo.st_size)
        raise Exception(
            'Failed to verify ' + filename + '. Can you get to it with a browser?')
    return filename

In [4]:
# Read the data into a list of strings.
def read_data(filename):
    """Extract the first file enclosed in a zip file as a list of words."""
    with zipfile.ZipFile(filename) as f:
        data = tf.compat.as_str(f.read(f.namelist()[0])).split()
    return data


In [5]:
def build_dataset(words, n_words):
    """Process raw inputs into a dataset."""
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(n_words - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

In [6]:
def collect_data(vocabulary_size=10000):
    url = 'http://mattmahoney.net/dc/'
    filename = maybe_download('text8.zip', url, 31344016)
    vocabulary = read_data(filename)
    print(vocabulary[:7])
    data, count, dictionary, reverse_dictionary = build_dataset(vocabulary,
                                                                vocabulary_size)
    del vocabulary  # Hint to reduce memory.
    return data, count, dictionary, reverse_dictionary

In [7]:
class SimilarityCallback:
    def run_sim(self):
        for i in range(valid_size):
            valid_word = reverse_dictionary[valid_examples[i]]
            top_k = 8  # number of nearest neighbors
            sim = self._get_sim(valid_examples[i])
            nearest = (-sim).argsort()[1:top_k + 1]
            log_str = 'Nearest to %s:' % valid_word
            for k in range(top_k):
                close_word = reverse_dictionary[nearest[k]]
                log_str = '%s %s,' % (log_str, close_word)
            print(log_str)

    @staticmethod
    def _get_sim(valid_word_idx):
        sim = np.zeros((vocab_size,))
        in_arr1 = np.zeros((1,))
        in_arr2 = np.zeros((1,))
        in_arr1[0,] = valid_word_idx
        for i in range(vocab_size):
            in_arr2[0,] = i
            out = validation_model.predict_on_batch([in_arr1, in_arr2])
            sim[i] = out
        return sim

In [18]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}

        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            # print(curr_word, line[1:])
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
            # break

    return words, word_to_vec_map

**Activations functions**

In [9]:
def relu(x):
    """
    Compute the relu of x

    Arguments:
    x -- A scalar or numpy array of any size.

    Return:
    s -- relu(x)
    """
    s = np.maximum(0,x)

    return s

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [10]:
def initialize_parameters(vocab_size, n_h):
    """
    Arguments:
    layer_dims -- python array (list) containing the dimensions of each layer in our network

    Returns:
    parameters -- python dictionary containing your parameters "W1", "b1", "W2", "b2":
                    W1 -- weight matrix of shape (n_h, vocab_size)
                    b1 -- bias vector of shape (n_h, 1)
                    W2 -- weight matrix of shape (vocab_size, n_h)
                    b2 -- bias vector of shape (vocab_size, 1)
    """

    np.random.seed(3)
    parameters = {}

    parameters['W1'] = np.random.randn(n_h, vocab_size) / np.sqrt(vocab_size)
    parameters['b1'] = np.zeros((n_h, 1))
    parameters['W2'] = np.random.randn(vocab_size, n_h) / np.sqrt(n_h)
    parameters['b2'] = np.zeros((vocab_size, 1))

    return parameters

## 2 - Load the Word Vectors

For this project, we will be loading 50 dimensional predtrained GloVe vectors to represent words.

In [19]:
words, word_to_vec_map = read_glove_vecs('glove.6B.50d.txt')

In [21]:
def cosine_similarity(u, v):
    '''
    Cosine similarity reflects the degree of similarity between u and v.

    Arguments:
        u -- a word vector of shape (n, )
        v -- a word vector of shape (n, )
    Returns:
        cosine_similarity -- the cosine similarity between u and v defined by the formula above.
    '''

    # Special case. Cosider the case u=[0, 0], v=[0, 0]
    if np.all(u==v):
        return 1

    # Compute the dot product between u and v
    dot = np.dot(u, v)

    # Compute the L2 norm of u
    norm_u = np.sqrt(np.sum(u*u))

    # Compute the L2 norm of v
    norm_v = np.sqrt(np.sum(v*v))

    # Avoid the division by 0
    if np.isclose(norm_u * norm_v, 0, atol=1e-32):
        return 0

    cosine_similarity = dot / (norm_u * norm_v)

    return cosine_similarity

In [25]:
# Test
father = word_to_vec_map["father"]
mother = word_to_vec_map["mother"]
ball = word_to_vec_map["ball"]
crocodile = word_to_vec_map["crocodile"]
france = word_to_vec_map["france"]
italy = word_to_vec_map["italy"]
paris = word_to_vec_map["paris"]
rome = word_to_vec_map["rome"]

print("cosine_similarity(father, mother) = ", cosine_similarity(father, mother))
print("cosine_similarity(ball, crocodile) = ",cosine_similarity(ball, crocodile))
print("cosine_similarity(france - paris, rome - italy) = ",cosine_similarity(france - paris, rome - italy))

# PUBLIC TESTS
def cosine_similarity_test(target):
    a = np.random.uniform(-10, 10, 10)
    b = np.random.uniform(-10, 10, 10)
    c = np.random.uniform(-1, 1, 23)

    assert np.isclose(cosine_similarity(a, a), 1), "cosine_similarity(a, a) must be 1"
    assert np.isclose(cosine_similarity((c >= 0) * 1, (c < 0) * 1), 0), "cosine_similarity(a, not(a)) must be 0"
    assert np.isclose(cosine_similarity(a, -a), -1), "cosine_similarity(a, -a) must be -1"
    assert np.isclose(cosine_similarity(a, b), cosine_similarity(a * 2, b * 4)), "cosine_similarity must be scale-independent. You must divide by the product of the norms of each input"

    print("\033[92mAll test passed!")

cosine_similarity_test(cosine_similarity)

cosine_similarity(father, mother) =  0.8909038442893615
cosine_similarity(ball, crocodile) =  0.2743924626137942
cosine_similarity(france - paris, rome - italy) =  -0.6751479308174201
All test passed!


In [27]:
def complete_analogy(word_a, word_b, word_c, word_to_vec_map):
    '''
    Performs the word analogy task as explained above: a is to b as c is to __.

    Arguments:
        word_a -- a word, string
        word_b -- a word, string
        word_c -- a word, string

    Returns:
        best_word -- the word such that v_b - v_a is close to
                     v_best_word - v_c, as measured by cosine similarity
    '''

    # convert words to lowercase
    word_a, word_b, word_c = word_a.lower(), word_b.lower(), word_c.lower()

    # Get the word embeddings e_a, e_b and e_c
    e_a, e_b, e_c = word_to_vec_map[word_a], word_to_vec_map[word_b], word_to_vec_map[word_c]

    words = word_to_vec_map.keys()
    max_cosine_sim = -100  # Initialize max_cosine_sim to a large negative number
    best_word = None       # Initialize best_word with None, it will help keep track of the word to output

    # Loop over the whole word vector set
    for w in words:
        # to avoid best_word being one the input words, skip the input word_c
        # skip word_c from query
        if w == word_c:
            continue

        # Compute the cosine similarity between the vector (e_b - e_a) and the vector
        # ((w's vector representation) - e_c)
        cosine_sim = cosine_similarity(e_b - e_a, word_to_vec_map[w] - e_c)

        # If the cosine_sim is more than the max_cosine_sim seen so far,
        # then: set the new max_cosine_sim to the current cosine_sim and the best_word
        # to the current word.
        if cosine_sim > max_cosine_sim:
            max_cosine_sim = cosine_sim
            best_word = w

    return best_word


In [28]:
# PUBLIC TEST
def complete_analogy_test(target):
    a    = [3, 3]   # Center at a
    a_nw = [2, 4]   # North-West oriented vector from a
    a_s  = [3, 2]   # South oriented vector from a

    c    = [-2, 1]  # Center at c

    # Create a controlled word to vec map
    word_to_vec_map = {'a': a,
                       'synonym_of_a': a,
                       'a_nw': a_nw,
                       'a_s' : a_s,
                       'c'   : c,
                       'c_n' : [-2, 2], # N
                       'c_ne': [-1, 2], # NE
                       'c_e' : [-1, 1], # E
                       'c_se': [-1, 0], # SE
                       'c_s' : [-2, 0], # S
                       'c_sw': [-3, 0], # SW
                       'c_w' : [-3, 1], # W
                       'c_nw': [-3, 2]  # NW
                      }

    # Convert lists to np.arrays
    for key in word_to_vec_map.keys():
        word_to_vec_map[key] = np.array(word_to_vec_map[key])

    assert(target('a', 'a_nw', 'c', word_to_vec_map) == 'c_nw')
    assert(target('a', 'a_s', 'c', word_to_vec_map) == 'c_s')
    assert(target('a', 'synonym_of_a', 'c', word_to_vec_map) != 'c'), "Best word cannot be input query"
    assert(target('a', 'c', 'a', word_to_vec_map) == 'c')

    print("\033[92mAll tests passed")

complete_analogy_test(complete_analogy)

All tests passed


In [31]:
triads_to_try = [('italy', 'italian', 'spain'),
                 ('india', 'delhi', 'japan'),
                 ('man', 'woman', 'boy'),
                 ('small', 'smaller', 'large'),
                 ('india', 'hindi', 'china'),
                 ('car', 'road', 'ship'),
                 ('pen', 'paper', 'keyboard')]

for triad in triads_to_try:
    print(f'{triad[0]} -> {triad[1]} :: {triad[2]} -> {complete_analogy(*triad, word_to_vec_map)}')

italy -> italian :: spain -> spanish
india -> delhi :: japan -> tokyo
man -> woman :: boy -> girl
small -> smaller :: large -> smaller
india -> hindi :: china -> hindi
car -> road :: ship -> river
pen -> paper :: keyboard -> materials


# Debiasing Word Vectors

In the following part of the project, we will examine gender biases that can be refelected in a word embedding, and explore algorthims for reducing the bias. In addition to the topic of biasing, this exercise will also help to get intution about what word vectors are doing.

First, see how the GloVe word embeddings relate to gender. We will begin by computing $g = e_{woman} - e_{man}$, where $e_{woman}$ represents the word vector corresponding to the word woman, and $e_{man}$ corresponds to the vector corresponding to the word man. The resulting vector $g$ roughly encodes the concept of "gender".

We might get a more accurate representation if we compute $g_1 = e_{mother}-e_{father}$, $g_2 = e_{girl} - e_{boy}$, etc. and average over them, but just using $e_{woman}-e_{man}$, will give good results for now.

In [32]:
g = word_to_vec_map['woman'] - word_to_vec_map['man']
print(g)

[-0.087144    0.2182     -0.40986    -0.03922    -0.1032      0.94165
 -0.06042     0.32988     0.46144    -0.35962     0.31102    -0.86824
  0.96006     0.01073     0.24337     0.08193    -1.02722    -0.21122
  0.695044   -0.00222     0.29106     0.5053     -0.099454    0.40445
  0.30181     0.1355     -0.0606     -0.07131    -0.19245    -0.06115
 -0.3204      0.07165    -0.13337    -0.25068714 -0.14293    -0.224957
 -0.149       0.048882    0.12191    -0.27362    -0.165476   -0.20426
  0.54376    -0.271425   -0.10245    -0.32108     0.2516     -0.33455
 -0.04371     0.01258   ]


Now, cosider the cosine similarity of different words with $g$. What does a positive value of similarity mean, versus a negative cosine similarity?

In [33]:
print ('List of names and their similarities with constructed vector:')

# girls and boys name
name_list = ['john', 'marie', 'sophie', 'ronaldo', 'priya', 'rahul', 'danielle', 'reza', 'katy', 'yasmin']

for w in name_list:
    print(w, cosine_similarity(word_to_vec_map[w], g))

List of names and their similarities with constructed vector:
john -0.23163356145973724
marie 0.315597935396073
sophie 0.31868789859418784
ronaldo -0.31244796850329437
priya 0.17632041839009402
rahul -0.16915471039231716
danielle 0.24393299216283895
reza -0.07930429672199553
katy 0.2831068659572615
yasmin 0.23313857767928758


As we can see, female first names tend to have a positive cosine similarity with our constructed vector $g$, while male first names tend to have a negative cosine similarity.

Let's try with some other words:

In [34]:
print('Other words and their similarities:')
word_list = ['lipstick', 'guns', 'science', 'arts', 'literature', 'warrior','doctor', 'tree', 'receptionist',
             'technology',  'fashion', 'teacher', 'engineer', 'pilot', 'computer', 'singer']
for w in word_list:
    print (w, cosine_similarity(word_to_vec_map[w], g))

Other words and their similarities:
lipstick 0.2769191625638267
guns -0.1888485567898898
science -0.06082906540929701
arts 0.008189312385880337
literature 0.06472504433459932
warrior -0.20920164641125288
doctor 0.11895289410935041
tree -0.07089399175478091
receptionist 0.33077941750593737
technology -0.13193732447554302
fashion 0.03563894625772699
teacher 0.17920923431825664
engineer -0.0803928049452407
pilot 0.0010764498991916937
computer -0.10330358873850498
singer 0.1850051813649629


It is astonishing to see that these results reflect certain unhealthy gender stereotypes. For example, we see "computer" is negative and is closer to in value to male first name, while "literature" is positive and is closer to female first names.



We will try to reduce the bias of these vectors, using an algorithm due to ```Boliukbasi, et al. 2016 ```. Note that some word pairs such as "actor"/"actress" or "grandmother"/"grandfather" should remain gender-specific, while other words such as "receptionist" or "technology" should be neutralized, i.e. not be gender-related. We will have to treat these two types of words differently when debiasing.  

### Neturalize Bias for Non-Gender specific Words



In [37]:
def neutralize(word, g, word_to_vec_map):
    '''
    Removes the bias of "word" by projecting it on the space orthogonal to the bias axis.
    This function ensures that gender neutral words are zero in the gender subspace.

    Arguments:
        word -- string indicating the word to debias
        g -- numpy-array of shape (50,) corresponding to the bias axis (such as gender)
        word_to_vec_map -- dictionary mapping words to their corresponding vectors.

    Returns:
        e_biased -- neutralized word vector representation of the input "word".
    '''

    # Select word vector representation of "word". Use word_to_vec_map
    e = word_to_vec_map[word]

    # Compute e_biascomponent using the formula given above.
    e_biascomponent = np.dot(e,g) / (np.sum(g*g)) * g

    # Neutralize e by substracting e_biascomponent from it
    # e_biased should be equal to its orthogonal projection.
    e_debiased = e - e_biascomponent

    return e_debiased

In [38]:
e = "receptionist"
print("cosine similarity between " + e + " and g, before neutralizing: ",
      cosine_similarity(word_to_vec_map["receptionist"], g))

e_debiased = neutralize("receptionist", g, word_to_vec_map)
print("cosine similarity between " + e + " and g, after neutralizing: ", cosine_similarity(e_debiased, g))

cosine similarity between receptionist and g, before neutralizing:  0.33077941750593737
cosine similarity between receptionist and g, after neutralizing:  -2.6832242276243644e-17


In [39]:
def equalize(pair, bias_axis, word_to_vec_map):
    '''
    Debias gender specific words by following the equalize method described in the figures above.

    Arguments:
        pair -- pair of strings of gender specific words to debias, e.g. ("actress", "actor")
        bias_axis -- numpy array of shape (50, ), vector corresponding to the bias axis, e.g. gender
        word_to_vec -- dictionary mapping words to their corresponding vectors

    Returns:
        e_1 -- word vector corresponding to the first word
        e_2 -- word vector corresponding to the second word
    '''

    # Step 1: Select word vector representation of "word". Use word_to_vec_map
    w1, w2 = pair
    e_w1, e_w2 = word_to_vec_map[w1], word_to_vec_map[w2]

    # Step 2: Compute the mean of e_w1 and e_w2
    mu = (e_w1 + e_w2) / 2

    # Step 3: Compute the projections of mu over the bias axis and the orthogonal axis
    mu_B = (mu * bias_axis) / np.sum(bias_axis*bias_axis) * bias_axis
    mu_orth = mu - mu_B

    # Step 4: Compute e_w1B and e_w2B
    e_w1B = np.dot(e_w1, bias_axis) / np.sum(bias_axis * bias_axis) * bias_axis
    e_w2B = np.dot(e_w2, bias_axis) / np.sum(bias_axis * bias_axis) * bias_axis

    # Step 5: Adjust the Bias part of e_w1B and e_w2B
    corrected_e_w1B = np.sqrt(abs(1-np.sum(mu_orth*mu_orth))) * (e_w1B - mu_B)/abs((e_w1 - mu_orth) - mu_B)
    corrected_e_w2B = np.sqrt(abs(1-np.sum(mu_orth*mu_orth))) * (e_w2B - mu_B)/abs((e_w2 - mu_orth) - mu_B)

    # Step 6: Debias by equalizing e1 and e2 to the sum of their corrected projections
    e1 = corrected_e_w1B + mu_orth
    e2 = corrected_e_w2B + mu_orth

    return e1, e2

In [40]:
print("cosine similarities before equalizing:")
print("cosine_similarity(word_to_vec_map[\"man\"], gender) = ", cosine_similarity(word_to_vec_map["man"], g))
print("cosine_similarity(word_to_vec_map[\"woman\"], gender) = ", cosine_similarity(word_to_vec_map["woman"], g))
print()
e1, e2 = equalize(("man", "woman"), g, word_to_vec_map)
print("cosine similarities after equalizing:")
print("cosine_similarity(e1, gender) = ", cosine_similarity(e1, g))
print("cosine_similarity(e2, gender) = ", cosine_similarity(e2, g))

cosine similarities before equalizing:
cosine_similarity(word_to_vec_map["man"], gender) =  -0.11711095765336832
cosine_similarity(word_to_vec_map["woman"], gender) =  0.35666618846270376

cosine similarities after equalizing:
cosine_similarity(e1, gender) =  -0.7195212663448881
cosine_similarity(e2, gender) =  0.7343917965586074
